<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>


## Introduction

<!--
In this lab, you will learn how to convert addresses into their equivalent latitude and longitude values. Also, you will use the Foursquare API to explore neighborhoods in New York City. You will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the *k*-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.
-->
This is the peer-reviewed assignment patterned after the Lab "Segmenting and Clustering Neighborhoods in New York City" but using the neighborhood data for Toronto. 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Toronto</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

<!--
Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572
-->
The dataset is obtained from City of Toronto's website at https://portal0.cf.opendata.inter.sandbox-toronto.ca/dataset/neighbourhoods/

Toronto has 140 neighborhood that are grouped into 25 Wards. The Wards are further grouped into 4 different community districts. This exercise uses the data of the 140 neighborhoods without further grouping them into the wards and districts.

The dataset is available as a GEOJSON file as well as a CSV file. the CSV file is used since it requires less manipulation.


In [2]:
# read the csv file downloaded from https://portal0.cf.opendata.inter.sandbox-toronto.ca/dataset/neighbourhoods/
df = pd.read_csv("Neighbourhoods.csv")

In [3]:
# Extracting the columns needed and renameing the columns
toronto_data=df[["AREA_NAME","AREA_DESC","LONGITUDE","LATITUDE"]]
new_columns = ["Neighborhood", "Code", "Longitude","Latitude"]
toronto_data.columns = new_columns
print("shape of data frame:", toronto_data.shape)
toronto_data.head()

shape of data frame: (140, 4)


,Neighborhood,Code,Longitude,Latitude
0,Wychwood (94),Wychwood (94),-79.425515,43.676919
1,Yonge-Eglinton (100),Yonge-Eglinton (100),-79.403590,43.704689
2,Yonge-St.Clair (97),Yonge-St.Clair (97),-79.397871,43.687859
3,York University Heights (27),York University Heights (27),-79.488883,43.765736
4,Yorkdale-Glen Park (31),Yorkdale-Glen Park (31),-79.457108,43.714672


In [6]:
# Spliting the neighborhood name form their code
toronto_data.loc[:,'Code'] = toronto_data.loc[:,'Code'].str.replace(".*\(","")
toronto_data.loc[:,'Code'] = toronto_data.loc[:,'Code'].str.replace("\)","")
toronto_data.loc[:,'Neighborhood'] = toronto_data.loc[:,'Neighborhood'].str.replace("\(.*\)","")
toronto_data.head()

,Neighborhood,Code,Longitude,Latitude
0,Wychwood,94,-79.425515,43.676919
1,Yonge-Eglinton,100,-79.403590,43.704689
2,Yonge-St.Clair,97,-79.397871,43.687859
3,York University Heights,27,-79.488883,43.765736
4,Yorkdale-Glen Park,31,-79.457108,43.714672


#### At this point, the data frame toronto_data is ready for use.

And make sure that the dataset has all 140 neighborhoods.

In [7]:
print('The dataframe has {} neighborhoods.'.format(
       toronto_data.shape[0]
    )
)

The dataframe has 140 neighborhoods.


#### Use geopy library to get the latitude and longitude values of city of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [8]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto  with neighborhoods superimposed on top.

In [9]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood.

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [10]:
CLIENT_ID = 'CLN1QMOSO2WMBTVJXM5CB2TBBBBRWEHS4A3ZPI25C045K3ZJ' # your Foursquare ID
CLIENT_SECRET = 'CNFHFFSJUGT0JZQT5PSQB4ERRJQGRU13IMRICEP31O2X10VU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CLN1QMOSO2WMBTVJXM5CB2TBBBBRWEHS4A3ZPI25C045K3ZJ
CLIENT_SECRET:CNFHFFSJUGT0JZQT5PSQB4ERRJQGRU13IMRICEP31O2X10VU


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [11]:
toronto_data.loc[0, 'Neighborhood']

'Wychwood '

Get the neighborhood's latitude and longitude values.

In [12]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude =toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wychwood  are 43.6769192679, -79.425514947.


#### Now, let's get the top 100 venues that are in Wychwood within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [13]:
# type your answer here
LIMIT = 100
radius = 500
latitude = neighborhood_latitude
longitude = neighborhood_longitude

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=CLN1QMOSO2WMBTVJXM5CB2TBBBBRWEHS4A3ZPI25C045K3ZJ&client_secret=CNFHFFSJUGT0JZQT5PSQB4ERRJQGRU13IMRICEP31O2X10VU&ll=43.6769192679,-79.425514947&v=20180605&radius=500&limit=100'

Send the GET request and examine the resutls

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d015598e97dfb002ca320ad'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bracondale Hill',
  'headerFullLocation': 'Bracondale Hill, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.6814192724, 'lng': -79.41930460333964},
   'sw': {'lat': 43.672419263399995, 'lng': -79.43172529066035}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b86e89df964a52051a531e3',
       'name': "Wychwood Barns Farmers' Market",
       'location': {'address': '601 Christie Street',
        'crossStreet': 'St Clair Avenue West',
        'lat': 43.68001040153905,
        'lng': -79.42384857341463,


From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wychwood Barns Farmers' Market,Farmers Market,43.680010,-79.423849
1,Wychwood Barns,Event Space,43.680028,-79.423810
2,Hillcrest Park,Park,43.676012,-79.424787
3,Annabelle Pasta Bar,Italian Restaurant,43.675445,-79.423341
4,Brewery Market,Beer Garden,43.679739,-79.423544


And how many venues were returned by Foursquare?

In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore All Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    i=1
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        print ("iteration:",i,name,"number of venues:", len(results))
        i=i+1
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [19]:
# type your answer here

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



iteration: 1 Wychwood  number of venues: 5
iteration: 2 Yonge-Eglinton  number of venues: 35
iteration: 3 Yonge-St.Clair  number of venues: 54
iteration: 4 York University Heights  number of venues: 12
iteration: 5 Yorkdale-Glen Park  number of venues: 24
iteration: 6 Lambton Baby Point  number of venues: 2
iteration: 7 Lansing-Westgate  number of venues: 1
iteration: 8 Lawrence Park North  number of venues: 48
iteration: 9 Lawrence Park South  number of venues: 8
iteration: 10 Leaside-Bennington  number of venues: 19
iteration: 11 Little Portugal  number of venues: 38
iteration: 12 Long Branch  number of venues: 14
iteration: 13 Malvern  number of venues: 6
iteration: 14 Maple Leaf  number of venues: 5
iteration: 15 Markland Wood  number of venues: 4
iteration: 16 Milliken  number of venues: 3
iteration: 17 Mimico  number of venues: 6
iteration: 18 Morningside  number of venues: 11
iteration: 19 Moss Park  number of venues: 41
iteration: 20 Mount Dennis  number of venues: 4
iteration:

#### Let's check the size of the resulting dataframe

In [20]:
print(toronto_venues.shape)

(2122, 7)


Let's check how many venues were returned for each neighborhood

In [21]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt North,20,20,20,20,20,20
Agincourt South-Malvern West,20,20,20,20,20,20
Alderwood,5,5,5,5,5,5
Annex,29,29,29,29,29,29
Banbury-Don Mills,21,21,21,21,21,21
Bathurst Manor,4,4,4,4,4,4
Bay Street Corridor,100,100,100,100,100,100
Bayview Village,3,3,3,3,3,3
Bayview Woods-Steeles,3,3,3,3,3,3


#### Let's find out how many unique categories can be curated from all the returned venues

In [22]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 280 uniques categories.


In [23]:
tdf = toronto_venues.groupby('Neighborhood').count()
tdf.shape

(137, 6)

Note: While Toronto has 140 neighborhoods,  3 of them had no nearby venus.  I added print out statement in the function to identify the three locations that have zero Venues. These are: *Newtonbrook West, St.Andrew-Windfields, and Willowridge-Martingrove-Richview.*

## 3. Analyze Each Neighborhood

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe - Note that "Neighborhood" is a category and there is alreay such a column
# I'm naming the new column NH_Name instead
toronto_onehot['NH_Name'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,NH_Name,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Stud

And let's examine the new dataframe size.

In [25]:
toronto_onehot.shape

(2122, 281)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
toronto_grouped = toronto_onehot.groupby('NH_Name').mean().reset_index()
toronto_grouped

,NH_Name,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Stud

#### Let's confirm the new size

In [27]:
toronto_grouped.shape

(137, 281)

#### Let's print each neighborhood along with the top 5 most common venues

In [28]:
num_top_venues = 5

for hood in toronto_grouped['NH_Name']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['NH_Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt North ----
                venue  freq
0  Chinese Restaurant  0.10
1        Liquor Store  0.05
2   Convenience Store  0.05
3      Clothing Store  0.05
4      Sandwich Place  0.05


----Agincourt South-Malvern West ----
                venue  freq
0  Chinese Restaurant  0.25
1       Shopping Mall  0.15
2  Seafood Restaurant  0.05
3    Malay Restaurant  0.05
4                Bank  0.05


----Alderwood ----
                     venue  freq
0              Pizza Place   0.4
1                 Pharmacy   0.2
2              Coffee Shop   0.2
3             Dance Studio   0.2
4  North Indian Restaurant   0.0


----Annex ----
            venue  freq
0            Café  0.10
1  Sandwich Place  0.10
2             Pub  0.07
3     Coffee Shop  0.07
4            Park  0.07


----Banbury-Don Mills ----
                 venue  freq
0           Shoe Store  0.10
1          Coffee Shop  0.10
2  Sporting Goods Shop  0.05
3         Liquor Store  0.05
4                  Spa  0.05


----Bathurst M

4   Falafel Restaurant  0.08


----Forest Hill North ----
                       venue  freq
0                Pizza Place  0.14
1  Middle Eastern Restaurant  0.05
2                       Park  0.05
3                        Spa  0.05
4         Frozen Yogurt Shop  0.05


----Forest Hill South ----
                venue  freq
0          Playground   0.5
1                Park   0.5
2  African Restaurant   0.0
3          Nail Salon   0.0
4        Noodle House   0.0


----Glenfield-Jane Heights ----
                     venue  freq
0                     Park  0.50
1                     Pool  0.25
2                 Tea Room  0.25
3  North Indian Restaurant  0.00
4                Nightclub  0.00


----Greenwood-Coxwell ----
                venue  freq
0   Indian Restaurant  0.19
1                Café  0.11
2       Grocery Store  0.05
3             Brewery  0.05
4  Italian Restaurant  0.03


----Guildwood ----
            venue  freq
0      Sports Bar  0.17
1     Pizza Place  0.17
2            

                     venue  freq
0              Bridal Shop   0.5
1                     Café   0.5
2             Optical Shop   0.0
3                   Office   0.0
4  North Indian Restaurant   0.0


----Niagara ----
        venue  freq
0        Park  0.12
1        Café  0.08
2  Restaurant  0.04
3  Nail Salon  0.04
4  Food Truck  0.04


----North Riverdale ----
                     venue  freq
0                     Park   0.4
1                     Café   0.4
2                     Pool   0.2
3             Optical Shop   0.0
4  North Indian Restaurant   0.0


----North St.James Town ----
           venue  freq
0    Coffee Shop  0.24
1  Grocery Store  0.09
2    Pizza Place  0.06
3     Steakhouse  0.03
4           Bank  0.03


----O'Connor-Parkview ----
            venue  freq
0     Pizza Place  0.15
1  Breakfast Spot  0.08
2        Pharmacy  0.08
3       Gastropub  0.08
4            Café  0.08


----Oakridge ----
                  venue  freq
0    Chinese Restaurant  0.17
1          Desse

                       venue  freq
0               Burger Joint  0.12
1                Pizza Place  0.12
2  Middle Eastern Restaurant  0.12
3                   Pharmacy  0.06
4                Coffee Shop  0.06


----Willowdale East ----
                     venue  freq
0      Dumpling Restaurant   0.5
1                    Hotel   0.5
2       African Restaurant   0.0
3             Neighborhood   0.0
4  North Indian Restaurant   0.0


----Willowdale West ----
                     venue  freq
0        Mobile Phone Shop   0.5
1                     Park   0.5
2             Optical Shop   0.0
3  North Indian Restaurant   0.0
4             Noodle House   0.0


----Woburn ----
                 venue  freq
0    Indian Restaurant  0.50
1  American Restaurant  0.25
2               Bakery  0.25
3         Neighborhood  0.00
4               Office  0.00


----Woodbine Corridor ----
            venue  freq
0  Baseball Field   0.2
1        Bus Stop   0.2
2            Café   0.2
3  Discount Store   0.2

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['NH_Name']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North,Chinese Restaurant,Pharmacy,Fried Chicken Joint,Cantonese Restaurant,Sandwich Place,Liquor Store,Japanese Restaurant,Clothing Store,Bakery,Discount Store
1,Agincourt South-Malvern West,Chinese Restaurant,Shopping Mall,Mediterranean Restaurant,Asian Restaurant,Motorcycle Shop,Malay Restaurant,Seafood Restaurant,Cantonese Restaurant,Noodle House,Bank
2,Alderwood,Pizza Place,Pharmacy,Dance Studio,Coffee Shop,Zoo Exhibit,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
3,Annex,Sandwich Place,Café,Pizza Place,Park,Pub,Coffee Shop,Social Club,French Restaurant,Pet Store,Pharmacy
4,Banbury-Don Mills,Shoe Store,Coffee Shop,Pet Store,Liquor Store,Sandwich Place,Chocolate Shop,Clothing Store,Italian Restaurant,Gourmet Shop,Cantonese Restaurant


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('NH_Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 2, 0, 1, 1, 1, 1, 1, 1, 3, 1, 1, 0,
       3, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 4, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)
#neighborhoods_venues_sorted

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


toronto_merged.head() # check the last columns!

,Neighborhood,Code,Longitude,Latitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wychwood,94,-79.425515,43.676919,1.0,Farmers Market,Beer Garden,Event Space,Italian Restaurant,Park,Farm,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Zoo Exhibit
1,Yonge-Eglinton,100,-79.403590,43.704689,1.0,Coffee Shop,Fast Food Restaurant,Gym,Japanese Restaurant,Smoothie Shop,Caribbean Restaurant,Poutine Place,Breakfast Spot,Tea Room,Italian Restaurant
2,Yonge-St.Clair,97,-79.397871,43.687859,1.0,Coffee Shop,Italian Restaurant,Pub,Pizza Place,Gym,Café,Sushi Restaurant,Bagel Shop,Pharmacy,Sandwich Place
3,York University Heights,27,-79.488883,43.765736,1.0,Pizza Place,Coffee Shop,Miscellaneous Shop,Massage Studio,Sushi Restaurant,Fast Food Restaurant,Falafel Restaurant,Caribbean Restaurant,Bar,Bank
4,Yorkdale-Glen Park,31,-79.457108,43.714672,1.0,Fast Food Restaurant,Restaurant,Italian Restaurant,Coffee Shop,Gym,Sandwich Place,Bookstore,Fried Chicken Joint,Bowling Alley,Discount Store


Notes:
Remember there are 3 Neighorhoods that have no nearby venues? they are not included in the clustering and their label has 
no a NaN value. As a result, the join made the reset of the values as float instead of int.

Below I put 5 as a new class (so we have 6 clusters!)  and convert the label value to integer. Integer is required since we use the cluster labels to select map colors

In [33]:
toronto_merged['Cluster_Labels'].fillna(5, inplace=True)
toronto_merged['Cluster_Labels'] = toronto_merged['Cluster_Labels'].astype('int32')
#toronto_merged.info()

Finally, let's visualize the resulting clusters

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

Less populated residential areas (suburb?) with parks, zoo exhibits, baseball fields, ethnic restaurants, farms......

In [35]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Lambton Baby Point,River,Park,Zoo Exhibit,Dumpling Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
20,Mount Olive-Silverstone-Jamestown,Coffee Shop,Japanese Restaurant,Park,Zoo Exhibit,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
27,North Riverdale,Café,Park,Pool,Dumpling Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
34,Parkwoods-Donalda,Food & Drink Shop,Park,Fast Food Restaurant,Farm,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
43,Rosedale-Moore Park,Playground,Park,Bank,Candy Store,Gym / Fitness Center,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
54,Taylor-Massey,Baseball Field,Theater,Park,Zoo Exhibit,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
62,West Hill,Park,Construction & Landscaping,Gym / Fitness Center,Tea Room,Falafel Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farm
64,Westminster-Branson,Construction & Landscaping,Gym / Fitness Center,Locksmith,Park,Zoo Exhibit,Farm,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
69,Willowdale West,Mobile Phone Shop,Park,Zoo Exhibit,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant
79,Bathurst Manor,Convenience Store,Park,Playground,Baseball Field,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm


#### Cluster 2
Downtown or dense residential areas with lots of restaurants/eateries, pub, grocery stores, markets, gym, malls, pharmacies, banks, ...

In [36]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wychwood,Farmers Market,Beer Garden,Event Space,Italian Restaurant,Park,Farm,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Zoo Exhibit
1,Yonge-Eglinton,Coffee Shop,Fast Food Restaurant,Gym,Japanese Restaurant,Smoothie Shop,Caribbean Restaurant,Poutine Place,Breakfast Spot,Tea Room,Italian Restaurant
2,Yonge-St.Clair,Coffee Shop,Italian Restaurant,Pub,Pizza Place,Gym,Café,Sushi Restaurant,Bagel Shop,Pharmacy,Sandwich Place
3,York University Heights,Pizza Place,Coffee Shop,Miscellaneous Shop,Massage Studio,Sushi Restaurant,Fast Food Restaurant,Falafel Restaurant,Caribbean Restaurant,Bar,Bank
4,Yorkdale-Glen Park,Fast Food Restaurant,Restaurant,Italian Restaurant,Coffee Shop,Gym,Sandwich Place,Bookstore,Fried Chicken Joint,Bowling Alley,Discount Store
7,Lawrence Park North,Bakery,Italian Restaurant,Coffee Shop,Sushi Restaurant,Tea Room,Burger Joint,Asian Restaurant,Bank,Sandwich Place,Frozen Yogurt Shop
8,Lawrence Park South,Sporting Goods Shop,Dessert Shop,Flower Shop,Chinese Restaurant,Gym / Fitness Center,Rental Car Location,Spa,Deli / Bodega,Egyptian Restaurant,Flea Market
9,Leaside-Bennington,Coffee Shop,Dessert Shop,Bike Shop,Fish & Chips Shop,Japanese Restaurant,Italian Restaurant,Shopping Mall,Sandwich Place,Skating Rink,Burger Joint
10,Little Portugal,Bar,Coffee Shop,Café,Cocktail Bar,Restaurant,Bakery,Dive Bar,Sports Bar,Boutique,Jazz Club
11,Long Branch,Coffee Shop,Pharmacy,Italian Restaurant,Bar,Greek Restaurant,Restaurant,Café,Beer Store,Pizza Place,Wings Joint


#### Cluster 3
Home of Construction & Landscaping, also with Fast Food, Farm, Electronic Stores, Egyptian Restaurants, Ethiopian Restaurant ...

In [37]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,Princess-Rosethorn,Construction & Landscaping,Playground,Farmers Market,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant
46,Rustic,Construction & Landscaping,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field,Eastern European Restaurant
84,Beechborough-Greenbrook,Construction & Landscaping,Sandwich Place,Fast Food Restaurant,Farm,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market


#### Cluster 4
Park & Zoo Exhibit, also with Egyptian Restaurant, Dumpling restaurant Ethiopian Restaurant, Electronic stores, Falafel restaurant, Framers Market...

In [38]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Lansing-Westgate,Park,Zoo Exhibit,Dumpling Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
15,Milliken,Park,Sandwich Place,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
92,Brookhaven-Amesbury,Park,Zoo Exhibit,Dumpling Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
96,Centennial Scarborough,Park,Zoo Exhibit,Dumpling Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market


#### Cluster 5
Neighborhood with baseball fields

In [39]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 4, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,Stonegate-Queensway,Baseball Field,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Zoo Exhibit,Egyptian Restaurant
125,Highland Creek,Bus Station,Home Service,Pharmacy,Neighborhood,Baseball Field,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant


#### Cluster 6
Don't forget the three neighborhoods that have no nearby venues!

In [40]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 5, toronto_merged.columns[[0] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Newtonbrook West,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,St.Andrew-Windfields,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,Willowridge-Martingrove-Richview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### This marks the end of the exercise